In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Drive import *

In [25]:
# Downloads the file from Gdrive
def fileDownloader():
    """Downloads Student Gradebook.xlsx from GDrive"""
    auth=GDriveAuthenticator.GDriveAuthenticator('Drive\\client_secret.json')
    drive=Gdrive.Gdrive(auth.authenticate())
    drive.download()

In [6]:
# imports the data and merges into one dataframe and returns it
def data_importer():
    """imports the data and merges into one dataframe and returns it"""
    excel_sheet=pd.read_excel('Student Gradebook.xlsx',sheet_name=None,usecols=range(0,10))
    data_july=excel_sheet['July'].copy()
    data_august=excel_sheet['August'].copy()
    data=pd.concat([data_july,data_august]).reset_index().drop(columns=['index'])
    del data_july
    del data_august
    return data

In [8]:
def mydayCleaner(data):
    idx=np.array([])
    for date,month in zip(data['Date'].values,data['Month'].values):
        date_mask=data['Date']==date
        month_mask=data['Month']==month
        task_mask=data['Task']=='AjKyaUkhada'
        d=data[date_mask & month_mask & task_mask].sort_values('Student')
        idx=np.append(idx,d[d.duplicated(subset=['Student'],keep='last')].index)
    data.drop(idx,inplace=True)
    return data

In [19]:
def knowledgeSharingCleaner(data):
    idx=np.array([])
    knowledge_sharing_days={'Monday':['Kunal','Bhavna','Apurwa'],\
                       'Tuesday':['Chandrima','Purbita','Prasoon'],\
                        'Wednesday':['Roumyak','Kaushal','Ujjainee'],\
                       'Thursday':['Siddhishikha','Dipam','Shakib'],\
                        'Friday':['Sonali','Durga','Sonali'],\
                       'Saturday':['Swaastick','Sharika','Vishal'],\
                       'Sunday':['Anjali','Arya','Surabhi']} 
    date_month_dayname_unique=data[['Date','Month','DayName']].drop_duplicates()
    for date,month,dayname in zip(date_month_dayname_unique['Date'].values.tolist(),date_month_dayname_unique['Month'].values.tolist(),date_month_dayname_unique['DayName'].values.tolist()):
        dateMask=data['Date']==date
        monthMask=data['Month']==month
        daynameMask=data['DayName']==dayname
        taskMask=data['Task']=='Knowledge Sharing'
        temp=data[dateMask & monthMask & daynameMask & taskMask]

        # Below for loop filters out those rows for which students and daynames dont match
        # Example:- if dipam posts on gyan on monday, then he wont be given marks
        for index,name in zip(temp.Student.index,temp.Student.values):
            if name not in knowledge_sharing_days[dayname]:
                idx=np.append(idx,index)
        # After the first loop is run (description given on top), now it's time to filter out
        # those who put more than one post in gyan channel on the same day
        idx=np.append(idx,temp[temp.duplicated(subset=['Student'],keep='last')].index)
    data.drop(idx,inplace=True)
    return data

In [20]:
def data_cleaner(data):
    """This method will clean the data thoroughly and return the cleaned dataframe"""
    data=data.dropna(how='all',axis=0)
    data.replace('Ajkyaukhada','AjKyaUkhada',inplace=True)
    data['Date']=pd.to_datetime(data['Date'],errors='ignore',infer_datetime_format=True,dayfirst=True)
    data.dropna(subset=['Student'],inplace=True)
    data.replace('Swaastik','Swaastick',inplace=True)
    data.replace('Sushree','Siddhishikha',inplace=True)
    niteshIndex=data[data['Student']=='Nitish'].index
    data.drop(niteshIndex,inplace=True)
    data['Month']=data['Date'].dt.month_name()
    data['Year']=data['Date'].dt.year
    data['DayName']=data['Date'].dt.weekday_name
    data['Date']=data['Date'].dt.day
    data=mydayCleaner(data)
    data=knowledgeSharingCleaner(data)
    del niteshIndex
    return data

In [21]:
data=data_importer()

In [22]:
data.shape

(1135, 10)

In [23]:
data=data_cleaner(data)

In [24]:
data.shape

(978, 13)

In [38]:
data

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Month,Year,DayName
0,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,July,2019,Monday
1,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,July,2019,Monday
2,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,July,2019,Monday
3,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,July,2019,Monday
4,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,July,2019,Monday
5,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kunal,0,8.0,10,0,July,2019,Monday
6,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Prasoon,0,5.0,10,0,July,2019,Monday
7,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Purbita,0,6.0,10,0,July,2019,Monday
8,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Roumyak,0,6.0,10,0,July,2019,Monday
9,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Shakib,0,6.0,10,0,July,2019,Monday


In [108]:
def get_myday(name,data,month_name=None):
    if month_name==None:
        myday='AjKyaUkhada'
        myday_marks=data[data['Task']==myday].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        temp=myday_marks[myday_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None
    else:
        myday='AjKyaUkhada'
        myday_marks=data[(data['Task']==myday) & (data['Month']==month_name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        temp=myday_marks[myday_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None

In [109]:
def get_task(name,data,month_name=None):
    knowledge='Knowledge Sharing'
    myday='AjKyaUkhada'
    if month_name==None:
        task_marks=data[(data['Task']!=knowledge) & (data['Task']!=myday)].groupby('Student')['Points'].\
        sum().sort_values(ascending=False).reset_index()
        temp=task_marks[task_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None
    else:
        task_marks=data[(data['Task']!=knowledge) & (data['Task']!=myday) & (data['Month']==month_name)].groupby('Student')['Points'].\
        sum().sort_values(ascending=False).reset_index()
        temp=task_marks[task_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None

In [110]:
def get_gyan(name,data,month_name=None):
    if month_name==None:
        knowledge_marks=data[data['Task']=='Knowledge Sharing'].groupby('Student')['Points'].sum().sort_values(ascending=False).\
        reset_index()
        temp=knowledge_marks[knowledge_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None
    else:
        knowledge_marks=data[(data['Task']=='Knowledge Sharing') & (data['Month']==month_name)].groupby('Student')['Points'].sum().sort_values(ascending=False).\
        reset_index()
        temp=knowledge_marks[knowledge_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return None

In [111]:
# Total marks function (whole)
def total_marks_leaderboard(data,month_name=None):
    if month_name==None:
        total_marks=data.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        total_marks.rename(columns={'Points':'Total Points'})
        total_marks['Task Marks']=total_marks['Student'].apply(get_task,args=[data])
        total_marks['MyDay Marks']=total_marks['Student'].apply(get_myday,args=[data])
        total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(get_gyan,args=[data])
    else:
        temp=data[data['Month']==month_name]
        total_marks=data[data['Month']==month_name].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        total_marks.rename(columns={'Points':'Total Points'})
        total_marks['Task Marks']=total_marks['Student'].apply(get_task,args=[temp,month_name])
        total_marks['MyDay Marks']=total_marks['Student'].apply(get_myday,args=[temp,month_name])
        total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(get_gyan,args=[temp,month_name])
    return total_marks

In [115]:
total_marks_leaderboard(data)

,Student,Points,Task Marks,MyDay Marks,Knowledge Sharing Marks
0,Kunal,252.5,150.5,96.0,6.0
1,Ujjainee,248.5,146.5,94.0,8.0
2,Shakib,239.5,141.5,90.0,8.0
3,Surabhi,239.5,143.5,94.0,2.0
4,Purbita,236.5,142.5,86.0,8.0
5,Vishal,231.5,139.5,90.0,2.0
6,Chandrima,227.0,125.0,92.0,10.0
7,Sonali,224.5,136.5,80.0,8.0
8,Dipam,221.5,125.5,94.0,2.0
9,Sharika,218.5,128.5,88.0,2.0


In [114]:
total_marks_leaderboard(data,month_name='August')

,Student,Points,Task Marks,MyDay Marks,Knowledge Sharing Marks
0,Kunal,126.5,60.5,62.0,4.0
1,Ujjainee,126.0,60.0,60.0,6.0
2,Surabhi,122.0,58.0,62.0,2.0
3,Shakib,121.5,57.5,58.0,6.0
4,Chandrima,120.0,52.0,62.0,6.0
5,Vishal,120.0,58.0,60.0,2.0
6,Purbita,115.0,57.0,54.0,4.0
7,Dipam,111.0,51.0,60.0,NaN
8,Durga,110.5,48.5,58.0,4.0
9,Siddhishikha,110.5,52.5,54.0,4.0


In [75]:
def student_task_marks(data,name=None,monthWise=False,month_name=None):
    """Returns the task and points of the given student name in DataFrame
    Arguments:
        data: dataframe from which marks need to be extracted
        name: name of the student, only the first name and first letter capital.
        monthWise: default: False
                    set to True if dataframe needed only for a particular month
        month_name: default None.
                    if monthWise True and month_name is name of the month then
                    Dataframe of that particular month is given.
                    
    Returns:
        dataframe
    """
    if name==None:
        raise ValueError('Input a Name')
    if monthWise==True and month_name!=None:
        task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') & (data['Student']==name) & \
                       (data['Month']==month_name)]
    elif monthWise==False:
        task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') & (data['Student']==name)]
    task_marks=task_marks[['Date','Month','Task','Student','Points','Total','Late Submission','Task Winner']].style.hide_index()
    return task_marks

In [76]:
student_task_marks(data,name='Kunal',monthWise=True,month_name='August')

Date,Month,Task,Student,Points,Total,Late Submission,Task Winner
1,August,Video Resume,Kunal,20,20,0,0
13,August,Github Profile Creation,Kunal,9.5,10,0,1
14,August,PPT Submission,Kunal,5,5,1,0
23,August,Blog on Data Analysis Process,Kunal,26,30,0,0


In [88]:
# Student myday count and marks
def student_myday(data,name=None,month_name=None):
    if name==None:
        raise ValueError('Input a name')
    if month_name==None:
        myday_marks=data[data['Task']==myday].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()